### Introduction

In [4]:
# Setup notebook
from pathlib import Path
from learntools.time_series.style import *  # plot style settings

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from statsmodels.tsa.deterministic import DeterministicProcess
from xgboost import XGBRegressor

store_sales = pd.read_csv(
    r'C:\Users\Genos\Projeler\Python İle Makine Öğrenmesi\Projeler\Store Sales\datasets\train.csv',
    usecols=['store_nbr', 'family', 'date', 'sales', 'onpromotion'],
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'sales': 'float32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
store_sales['date'] = store_sales.date.dt.to_period('D')
store_sales = store_sales.set_index(['store_nbr', 'family', 'date']).sort_index()

family_sales = (
    store_sales
        .groupby(['family', 'date'])
        .mean()
        .unstack('family')
        .loc['2017']
)

store_sales.unstack()

sales                           \
date                                 2013-01-01   2013-01-02  2013-01-03   
store_nbr family                                                           
1         AUTOMOTIVE                        0.0     2.000000    3.000000   
          BABY CARE                         0.0     0.000000    0.000000   
          BEAUTY                            0.0     2.000000    0.000000   
          BEVERAGES                         0.0  1091.000000  919.000000   
          BOOKS                             0.0     0.000000    0.000000   
...                                         ...          ...         ...   
9         POULTRY                           0.0   651.291992  509.496002   
          PREPARED FOODS                    0.0    83.000000   66.000000   
          PRODUCE                           0.0     0.000000    0.000000   
          SCHOOL AND OFFICE SUPPLIES        0.0     0.000000    0.000000   
          SEAFOOD                           0.0    29.214001   25.000000   

                                                                           \
date                                  2013-01-04   2013-01-05  2013-01-06   
store_nbr family                                                            
1         AUTOMOTIVE                    3.000000     5.000000    2.000000   
          BABY CARE                     0.000000     0.000000    0.000000   
          BEAUTY                        3.000000     3.000000    0.000000   
          BEVERAGES                   953.000000  1160.000000  407.000000   
          BOOKS                         0.000000     0.000000    0.000000   
...                                          ...          ...         ...   
9         POULTRY                     332.671997   510.919006  604.244019   
          PREPARED FOODS               57.000000    84.000000   82.000000   
          PRODUCE                       0.000000     0.000000    0.000000   
          SCHOOL AND OFFICE SUPPLIES    0.000000     0.000000    0.000000   
          SEAFOOD                      11.000000    28.361000   36.553001   

                                                                            \
date                                   2013-01-07   2013-01-08  2013-01-09   
store_nbr family                                                             
1         AUTOMOTIVE                     0.000000     2.000000     2.00000   
          BABY CARE                      0.000000     0.000000     0.00000   
          BEAUTY                         1.000000     3.000000     1.00000   
          BEVERAGES                   1158.000000  1029.000000  1186.00000   
          BOOKS                          0.000000     0.000000     0.00000   
...                                           ...          ...         ...   
9         POULTRY                      435.082001   372.350983   317.42099   
          PREPARED FOODS                31.000000    26.000000    24.00000   
          PRODUCE                        0.000000     0.000000     0.00000   
          SCHOOL AND OFFICE SUPPLIES     0.000000     0.000000     0.00000   
          SEAFOOD                       18.444000    25.000000    13.48400   

                                                  ... onpromotion             \
date                                  2013-01-10  ...  2017-08-06 2017-08-07   
store_nbr family                                  ...                          
1         AUTOMOTIVE                    2.000000  ...           0          0   
          BABY CARE                     0.000000  ...           0          0   
          BEAUTY                        3.000000  ...           1          1   
          BEVERAGES                   847.000000  ...           5          7   
          BOOKS                         0.000000  ...           0          0   
...                                          ...  ...         ...        ...   
9         POULTRY                     351.544006  ...           0          0   
     

In [10]:
family_sales

sales                                              \
family     AUTOMOTIVE BABY CARE     BEAUTY    BEVERAGES     BOOKS   
date                                                                
2017-01-01   0.092593  0.037037   0.055556    74.222221  0.000000   
2017-01-02  11.481482  0.259259  11.648149  6208.055664  0.481481   
2017-01-03   8.296296  0.296296   7.185185  4507.814941  0.814815   
2017-01-04   6.833333  0.333333   6.888889  3911.833252  0.759259   
2017-01-05   6.333333  0.351852   5.925926  3258.796387  0.407407   
...               ...       ...        ...          ...       ...   
2017-08-11   8.166667  0.129630   6.500000  3502.055664  0.000000   
2017-08-12   7.462963  0.055556   6.833333  3376.259277  0.000000   
2017-08-13   8.907408  0.166667   8.018518  3747.296387  0.018519   
2017-08-14   5.407407  0.166667   6.240741  3237.629639  0.000000   
2017-08-15   6.240741  0.148148   6.277778  3162.462891  0.000000   

                                                                           \
family     BREAD/BAKERY CELEBRATION     CLEANING        DAIRY        DELI   
date                                                                        
2017-01-01     9.084685    0.129630     7.500000    11.518518    3.629167   
2017-01-02   844.836304   14.203704  2233.648193  1545.000000  539.114807   
2017-01-03   665.124084   10.629630  1711.907349  1204.203735  404.300079   
2017-01-04   594.160583   11.185185  1508.036987  1107.796265  309.397675   
2017-01-05   495.511597   12.444445  1241.833374   829.277771  260.776489   
...                 ...         ...          ...          ...         ...   
2017-08-11   522.305298   16.111111  1031.148193   831.648132  345.934967   
2017-08-12   507.473114   15.722222  1072.870361   793.592590  290.553589   
2017-08-13   615.976990   10.074074  1143.648193   928.000000  325.801361   
2017-08-14   518.564026   11.000000  1019.111084   797.222229  271.532227   
2017-08-15   539.966553   11.055555  1082.851807   753.833313  276.582458   

            ... onpromotion                                        \
family      ...   MAGAZINES      MEATS PERSONAL CARE PET SUPPLIES   
date        ...                                                     
2017-01-01  ...         0.0   0.018519      0.111111     0.018519   
2017-01-02  ...         0.0   0.462963     10.592593     0.537037   
2017-01-03  ...         0.0   0.481481      9.722222     0.444444   
2017-01-04  ...         0.0   0.370370     12.037037     0.444444   
2017-01-05  ...         0.0   8.981481      5.666667     0.000000   
...         ...         ...        ...           ...          ...   
2017-08-11  ...         0.0  22.925926      9.000000     0.000000   
2017-08-12  ...         0.0   0.000000      8.703704     0.000000   
2017-08-13  ...         0.0   0.000000      9.240741     0.000000   
2017-08-14  ...         0.0   0.000000      8.722222     0.000000   
2017-08-15  ...         0.0   0.000000      8.814815     0.000000   

                                                                          \
family     PLAYERS AND ELECTRONICS    POULTRY PREPARED FOODS     PRODUCE   
date                                                                       
2017-01-01                0.000000   0.000000       0.037037    0.129630   
2017-01-02                0.000000   0.259259       1.166667    5.629630   
2017-01-03                0.000000   0.388889       1.351852   56.296296   
2017-01-04                0.000000   0.296296       5.444444  101.277778   
2017-01-05                0.000000   0.296296       0.907407    5.018519   
...                            ...        ...            ...         ...   
2017-08-11                0.000000  27.407407       0.537037    6.000000   
2017-08-12                0.018519   0.111111       0.277778    6.000000   
2017-08-13                0.000000   0.185185       0.222222    5.907407   
2017-08-14                0.000000   0.129630       0.148148    6.018519   
2017-08-15     

Sonraki iki soruda, yeni bir Python sınıfı uygulayarak Store Sales için boosted hybrid oluşturacağız.
İlk sınıf tanımını oluşturmak için bu hücreyi çalıştırın. Scikit-learn benzeri bir arayüz vermek için uygun ve tahmin yöntemleri ekleyeceksiniz.

In [ ]:
# You'll add fit and predict methods to this minimal class
class BoostedHybrid:
    def __init__(self, model_1, model_2):
        self.model_1 = model_1
        self.model_2 = model_2
        self.y_columns = None  # store column names from fit method

### 1) Define fit method for boosted hybrid (Güçlendirilmiş hibrit için uyum yöntemini tanımlayın)

BoostedHybrid sınıfı için uygunluk tanımını tamamlayın. Gerekirse öğreticideki Hybrid Forecasting with Residuals bölümündeki 1. ve 2. adımlara geri dönün.

In [ ]:
def fit(self, X_1, X_2, y):
    # YOUR CODE HERE: fit self.model_1
    self.model_1.fit(X_1, y)

    # Make predictions with self.model_1
    y_fit = pd.DataFrame(
        # YOUR CODE HERE: make predictions with self.model_1
        self.model_1.predict(X_1),
        index=X_1.index, columns=y.columns,
    )

    # YOUR CODE HERE: compute residuals
    y_resid = y - y_fit
    y_resid = y_resid.stack().squeeze()  # wide to long

    # YOUR CODE HERE: fit self.model_2 on residuals
    self.model_2.fit(X_2, y_resid)

    # Save column names for predict method
    self.y_columns = y.columns
    # Save data for question checking
    self.y_fit = y_fit
    self.y_resid = y_resid


# Add method to class
BoostedHybrid.fit = fit

### 2) Define predict method for boosted hybrid (2) Güçlendirilmiş hibrit için tahmin yöntemini tanımlayın)

Şimdi BoostedHybrid sınıfı için tahmin yöntemini tanımlayın. Gerekirse öğreticideki Hybrid Forecasting with Residuals 3. adıma geri dönün.

In [ ]:
def predict(self, X_1, X_2):
    y_pred = pd.DataFrame(
        # YOUR CODE HERE: predict with self.model_1
        self.model_1.predict(X_1),
        index=X_1.index, columns=self.y_columns,
    )
    y_pred = y_pred.stack().squeeze()  # wide to long

    # YOUR CODE HERE: add self.model_2 predictions to y_pred
    y_pred += self.model_2.predict(X_2)

    return y_pred.unstack()  # long to wide


# Add method to class
BoostedHybrid.predict = predict

Artık Mağaza Satışları verileri için bir model oluşturmak üzere yeni BoostedHybrid sınıfınızı kullanmaya hazırsınız. Eğitim için verileri ayarlamak için sonraki hücreyi çalıştırın.

In [ ]:
# Target series
y = family_sales.loc[:, 'sales']

# X_1: Features for Linear Regression
dp = DeterministicProcess(index=y.index, order=1)
X_1 = dp.in_sample()

# X_2: Features for XGBoost
X_2 = family_sales.drop('sales', axis=1).stack()  # onpromotion feature

# Label encoding for 'family'
le = LabelEncoder()  # from sklearn.preprocessing
X_2 = X_2.reset_index('family')
X_2['family'] = le.fit_transform(X_2['family'])

# Label encoding for seasonality
X_2["day"] = X_2.index.day  # values are day of the month

X_2

### 3) Train boosted hybrid


LinearRegression() ve XGBRegressor() örnekleriyle bir BoostedHybrid sınıfını başlatarak hibrit modeli oluşturun.

In [ ]:
# YOUR CODE HERE: Create LinearRegression + XGBRegressor hybrid with BoostedHybrid
model = BoostedHybrid(
    model_1=LinearRegression(),
    model_2=XGBRegressor(),
)

# YOUR CODE HERE: Fit and predict
model.fit(X_1, X_2, y)
y_pred = model.predict(X_1, X_2)

y_pred = y_pred.clip(0.0)

Sorununuza bağlı olarak, önceki sorularda oluşturduğunuz lineer regresyon + XGBoost hibriti dışında başka hibrit kombinasyonları kullanmak isteyebilirsiniz. scikit-learn'den diğer algoritmaları denemek için sonraki hücreyi çalıştırın.

In [ ]:
# Model 1 (trend)
from pyearth import Earth
from sklearn.linear_model import ElasticNet, Lasso, Ridge

# Model 2
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

# Boosted Hybrid

# YOUR CODE HERE: Try different combinations of the algorithms above
model = BoostedHybrid(
    model_1=Ridge(),
    model_2=KNeighborsRegressor(),
)

Bunlar sadece bazı öneriler. scikit-learn [Kullanıcı Kılavuzunda](https://scikit-learn.org/stable/supervised_learning.html) beğendiğiniz diğer algoritmaları keşfedebilirsiniz.

Hybrid modellerin yaptığı tahminleri görmek için bu hücredeki kodu kullanın.

In [ ]:
y_train, y_valid = y[:"2017-07-01"], y["2017-07-02":]
X1_train, X1_valid = X_1[: "2017-07-01"], X_1["2017-07-02":]
X2_train, X2_valid = X_2[: "2017-07-01"], X_2["2017-07-02":]

# Some of the algorithms above do best with certain kinds of
# preprocessing on the features (like standardization), but this is
# just a demo.
model.fit(X1_train, X2_train, y_train)
y_fit = model.predict(X1_train, X2_train).clip(0.0)
y_pred = model.predict(X1_valid, X2_valid).clip(0.0)

families = y.columns[0:6]
axs = y.loc(axis=1)[families].plot(
    subplots=True, sharex=True, figsize=(11, 9), **plot_params, alpha=0.5,
)
_ = y_fit.loc(axis=1)[families].plot(subplots=True, sharex=True, color='C0', ax=axs)
_ = y_pred.loc(axis=1)[families].plot(subplots=True, sharex=True, color='C3', ax=axs)
for ax, family in zip(axs, families):
    ax.legend([])
    ax.set_ylabel(family)